In [94]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


df3=pd.read_csv('pupfile_ready402.xls','\t') 
print df3.batch.count()
dummy_ac=pd.get_dummies(df3.Age_Cat, prefix='agecat')
print dummy_ac.describe()
del dummy_ac['agecat_Baby']
df3 = df3.join(dummy_ac)



df3['shortlist'] = [1 if x<40 else 0 for x in df3['days_on_site']]
print df3['shortlist'].describe()

6695
       agecat_Adult  agecat_Baby  agecat_Senior  agecat_Young
count   6695.000000  6695.000000    6695.000000   6695.000000
mean       0.497386     0.123226       0.082599      0.296789
std        0.500031     0.328721       0.275296      0.456877
min        0.000000     0.000000       0.000000      0.000000
25%        0.000000     0.000000       0.000000      0.000000
50%        0.000000     0.000000       0.000000      0.000000
75%        1.000000     0.000000       0.000000      1.000000
max        1.000000     1.000000       1.000000      1.000000
count    6695.000000
mean        0.924720
std         0.263862
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: shortlist, dtype: float64


In [95]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
count_vect = CountVectorizer(stop_words='english', max_features=20)
breed_words = count_vect.fit_transform(df3['Breed']).toarray()
print breed_words.shape
print breed_words[4]

bw=pd.DataFrame(data=breed_words, columns=count_vect.get_feature_names())
df=df3[['shortlist','sentiment','days_on_site','Description']].join(bw)


(6695, 20)
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]


In [96]:
df.head()

,shortlist,sentiment,days_on_site,Description,american,australian,boxer,bull,bulldog,chihuahua,...,miniature,mix,mutt,pit,poodle,retriever,shepherd,staffordshire,terrier,unknown
0,1,0.174038,0,Adopting A Shelter Dog These dogs come without...,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,1,0.258461,0,Please contact Denise (ramonahumanesociety@yah...,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0.313317,0,Meet Willow! She is an adorable loving 3 year ...,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
3,1,0.443125,0,www.fieldofdreamrescue.net Email; fieldofdream...,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
4,1,0.280000,0,United Hope for Animals (UHA) is a volunteer g...,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [97]:
df=df[df.Description.notnull()]
Q1 = df.sentiment.quantile(0.25)
Q3 = df.sentiment.quantile(0.75)
IQR = Q3 - Q1
df.drop(df[df['sentiment'] > (Q3 + 1.5*IQR)].index, inplace = True)
df.drop(df[df['sentiment'] < (Q1 - 1.5*IQR)].index, inplace = True)
print df.sentiment.describe()
df['sense']=[0 if x<.241 else 1 for x in df['sentiment']]

count    5213.000000
mean        0.238949
std         0.100025
min        -0.035556
25%         0.172225
50%         0.235185
75%         0.302874
max         0.510000
Name: sentiment, dtype: float64


In [98]:
df.sense.describe()

count    5213.000000
mean        0.477268
std         0.499531
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: sense, dtype: float64

In [99]:
df.columns.values

array(['shortlist', 'sentiment', 'days_on_site', 'Description',
       u'american', u'australian', u'boxer', u'bull', u'bulldog',
       u'chihuahua', u'dachshund', u'dog', u'german', u'labrador',
       u'miniature', u'mix', u'mutt', u'pit', u'poodle', u'retriever',
       u'shepherd', u'staffordshire', u'terrier', u'unknown', 'sense'], dtype=object)

In [100]:
df=df.drop(['Description','days_on_site'],axis=1)
print df.columns.values
y=df.sense
X=df.copy()
X=X.drop(['shortlist','sentiment','sense'], axis=1)
X.columns.values

['shortlist' 'sentiment' u'american' u'australian' u'boxer' u'bull'
 u'bulldog' u'chihuahua' u'dachshund' u'dog' u'german' u'labrador'
 u'miniature' u'mix' u'mutt' u'pit' u'poodle' u'retriever' u'shepherd'
 u'staffordshire' u'terrier' u'unknown' 'sense']


array([u'american', u'australian', u'boxer', u'bull', u'bulldog',
       u'chihuahua', u'dachshund', u'dog', u'german', u'labrador',
       u'miniature', u'mix', u'mutt', u'pit', u'poodle', u'retriever',
       u'shepherd', u'staffordshire', u'terrier', u'unknown'], dtype=object)

In [101]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier  #VotingClassifier is part of sklearn.ensemble


In [102]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(max_depth=5, n_estimators = 3000)
clf3 = BernoulliNB()
print y.mean()

0.477268367543


In [103]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3)], voting='hard')
for clf in [clf1,clf2,clf3,eclf]:
    scores = cross_validation.cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print scores.mean()

0.54324724806
0.560718887197
0.546330810922
0.547284252947


In [104]:
clf1.fit(X,y)
L=X.columns.values
for i in range(0,L.size):
    print L[i], clf1.coef_[0][i], np.exp(clf1.coef_[0][i]) - 1


american 0.150879268604 0.162856256348
australian 0.209864408783 0.233510795387
boxer 0.25358858199 0.28864152486
bull -0.274772750958 -0.240255245123
bulldog 0.134717782773 0.144213821929
chihuahua -0.404887814257 -0.332948359655
dachshund 0.323712047147 0.3822492272
dog -0.626845029312 -0.465725234016
german 0.499033362951 0.64712832566
labrador 0.3146286299 0.369750532098
miniature 0.0397036999804 0.0405024276234
mix 0.000838631408612 0.000838983158254
mutt 0.322279827105 0.38027095915
pit 0.467316656371 0.59570661409
poodle 0.0739092606891 0.0767091012216
retriever -0.00351204856135 -0.00350588853235
shepherd -0.0614876696606 -0.0596354593181
staffordshire 0.467042656607 0.595269450748
terrier 0.124221775639 0.132266952383
unknown 0.122058017331 0.129819648999


In [105]:
from sklearn.metrics import confusion_matrix
y_hat = clf1.predict(X)
confusion_matrix(y,y_hat)


array([[1634, 1091],
       [1153, 1335]])

In [106]:
X1=pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]],columns=L)
X1

,american,australian,boxer,bull,bulldog,chihuahua,dachshund,dog,german,labrador,miniature,mix,mutt,pit,poodle,retriever,shepherd,staffordshire,terrier,unknown
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [107]:
X1.pit=1
X1.bull=1
X1.american=1
X1.terrier=1

In [108]:
print clf1.predict(X1)
print(clf1.predict_proba(X1))

[1]
[[ 0.45713552  0.54286448]]


In [109]:
X1=pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]],columns=L)
X1.chihuahua=1
print clf1.predict(X1)
print(clf1.predict_proba(X1))

[0]
[[ 0.66833252  0.33166748]]


In [110]:
df.sentiment.describe()

count    5213.000000
mean        0.238949
std         0.100025
min        -0.035556
25%         0.172225
50%         0.235185
75%         0.302874
max         0.510000
Name: sentiment, dtype: float64

In [111]:
# mean .241, median .236
plt.plot(df)

AttributeError: 'module' object has no attribute 'plot'

In [ ]:
import matplotlib.pyplot as pyp
%matplotlib notebook


In [ ]:
df['sentiment'].plot(kind='box')

In [ ]:
#Step 2 Let's find IQR
Q1 = df.sentiment.quantile(0.25)
Q3 = df.sentiment.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
df.drop(df[df['sentiment'] > (Q3 + 1.5*IQR)].index, inplace = True)
df.drop(df[df['sentiment'] < (Q1 - 1.5*IQR)].index, inplace = True)

In [ ]:
df.describe()